In [17]:
import scipy.sparse as sp
import pandas as pd
import numpy as np


class Dataset(object):
    def __init__(self, path):
        self.trainMatrix = self.load_rating_file_as_matrix(path + ".train.rating")
        self.testRatings = self.load_rating_file_as_list(path + ".test.rating")
        self.testNegatives = self.load_negative_file(path + ".test.negative")
        assert len(self.testRatings) == len(self.testNegatives)
        
        self.num_users, self.num_items = self.trainMatrix.shape

    def load_rating_file_as_list(self, filename):
        df = pd.read_csv(filename, sep="\t")
        ratingList = list(zip(df.userid.tolist(), df.itemid.tolist()))
        return ratingList
    
    def load_negative_file(self, filename):
        df = pd.read_csv(filename, sep="\t")
        negativeList = df.iloc[:, 1:].values.tolist()
        return negativeList
    
    def load_rating_file_as_matrix(self, filename):
        df = pd.read_csv(filename, sep="\t")
        num_users = df.userid.max()
        num_items = df.itemid.max()
        mat = sp.dok_matrix((num_users + 1, num_items + 1), dtype=np.float32)
        interactions = df[['userid', 'itemid']].values.tolist()
        # [(0, 2969), (0, 1178), (0, 1574), (0, 957)]
        for user, item in interactions:
            mat[user, item] = 1.
        # [((0, 2969), 1.0), ((0, 1178), 1.0), ((0, 1574), 1.0), ((0, 957), 1.0)]
        return mat

In [18]:
import math
import heapq
import numpy as np


def evaluate_model(model, testRatings, testNegatives, topK):

    global _model
    global _testRatings
    global _testNegatives
    global _topK

    _model = model
    _testRatings = testRatings
    _testNegatives = testNegatives
    _topK = topK

    hits, ndcgs = [],[]
    for idx in range(len(_testRatings)):
        (hr,ndcg) = eval_one_rating(idx)
        hits.append(hr)
        ndcgs.append(ndcg)
    return (hits, ndcgs)


def eval_one_rating(idx):
    rating = _testRatings[idx]
    items = _testNegatives[idx]
    u = rating[0]
    gtItem = rating[1]
    items.append(gtItem)
    map_item_score = {}
    users = np.full(len(items), u, dtype = 'int32')
    predictions = _model.predict([users, np.array(items)],
                                 batch_size=100, verbose=0)

    for i in range(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i]
    items.pop()

    ranklist = heapq.nlargest(_topK, map_item_score, key=map_item_score.get)
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    return (hr, ndcg)


def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0


def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

def get_train_instances(train, n_items, n_neg, testNegatives):
    user, item, labels = [],[],[]
    n_users = train.shape[0]
    for (u, i) in train.keys():
        # positive instance
        user.append(u)
        item.append(i)
        labels.append(1)
        # negative instances: we also need to make sure they are not in the
        # test dataset
        for t in range(n_neg):
            j = np.random.randint(n_items)
            while ((u, j) in train.keys()) or (j in testNegatives[u]):
                j = np.random.randint(n_items)
            user.append(u)
            item.append(j)
            labels.append(0)
    return np.array(user), np.array(item), np.array(labels)

In [19]:
import numpy as np
import pandas as pd
import os
import heapq
import keras
import multiprocessing as mp
from keras import initializers
from keras.models import Model, load_model, save_model
from keras.layers import Dense, Embedding, Input, merge, Flatten, multiply
from keras.optimizers import Adagrad, Adam, SGD, RMSprop
from keras.regularizers import l2

from time import time

def GMF(n_users, n_items, n_emb, reg):
    user = Input(shape=(1,), dtype='int32', name='user_input')
    item = Input(shape=(1,), dtype='int32', name='item_input')

    # user and item embeddings
    # [?, 1, 8]
    MF_Embedding_User = Embedding(
        input_dim=n_users,
        output_dim=n_emb,
        name='user_embedding',
        embeddings_initializer='normal',
        embeddings_regularizer=l2(reg),
        input_length=1)
    # [?, 1, 8]
    MF_Embedding_Item = Embedding(
        input_dim=n_items,
        output_dim=n_emb,
        name='item_embedding',
        embeddings_initializer='normal',
        embeddings_regularizer=l2(reg),
        input_length=1)

    # Flatten and multiply
    # [?, 8]
    user_latent = Flatten()(MF_Embedding_User(user))
    # [?, 8]
    item_latent = Flatten()(MF_Embedding_Item(item))
    # [?, 8]
    predict_vector = multiply([user_latent, item_latent])

    # output layer
    prediction = Dense(1, activation='sigmoid',
                       kernel_regularizer=l2(reg),
                       kernel_initializer='lecun_uniform',
                       name='prediction')(predict_vector)

    # Model
    model = Model(inputs=[user, item], outputs=prediction)

    return model


if __name__ == '__main__':
    datadir = "Data_Javier/"
    dataname = "ml-1m"
    modeldir = "models"
    n_emb = 8
    reg = 0.0
    batch_size = 256
    epochs = 20
    learner = "adam"
    lr = 0.01
    validate_every = 1
    save_model = True
    topK = 10
    n_neg = 4

    modelfname = "keras_GMF" + \
                 "_".join(["_bs", str(batch_size)]) + \
                 "_".join(["_reg", str(reg).replace(".", "")]) + \
                 "_".join(["_lr", str(lr).replace(".", "")]) + \
                 "_".join(["_n_emb", str(n_emb)]) + ".h5"
    modelpath = os.path.join(modeldir, modelfname)
    resultsdfpath = os.path.join(modeldir, 'results_df.p')

    dataset = Dataset(os.path.join(datadir, dataname))
    train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
    n_users, n_items = train.shape

    model = GMF(n_users, n_items, n_emb, reg)
    if learner.lower() == "adagrad":
        model.compile(optimizer=Adagrad(lr=lr), loss='binary_crossentropy')
    elif learner.lower() == "rmsprop":
        model.compile(optimizer=RMSprop(lr=lr), loss='binary_crossentropy')
    elif learner.lower() == "adam":
        model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy')
    else:
        model.compile(optimizer=SGD(lr=lr), loss='binary_crossentropy')

    best_hr, best_ndcg, best_iter = 0, 0, 0
    for epoch in range(1, epochs + 1):
        t1 = time()
        user, item, labels = get_train_instances(train, n_items, n_neg, testNegatives)
        hist = model.fit([user, item], labels, batch_size=batch_size, epochs=1, verbose=0, shuffle=True)
        t2 = time()
        if epoch % validate_every == 0:
            (hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK)
            hr, ndcg, loss = np.array(hits).mean(), np.array(ndcgs).mean(), hist.history['loss'][0]
            print("Iteration {}: {:.2f}s, HR = {:.4f}, NDCG = {:.4f}, loss = {:.4f}, validated in {:.2f}s"
                  .format(epoch, t2 - t1, hr, ndcg, loss, time() - t2))
            if hr > best_hr:
                best_hr, best_ndcg, best_iter, train_time = hr, ndcg, epoch, t2 - t1
                if save_model:
                    model.save_weights(modelpath, overwrite=True)

    print("End. Best Iteration {}:  HR = {:.4f}, NDCG = {:.4f}. "
          .format(best_iter, best_hr, best_ndcg))
    if save_model:
        print("The best GMF model is saved to {}".format(modelpath))







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








Iteration 1: 55.03s, HR = 0.5674, NDCG = 0.3135, loss = 0.3312, validated in 4.73s
Iteration 2: 54.67s, HR = 0.5826, NDCG = 0.3248, loss = 0.3019, validated in 4.67s
Iteration 3: 51.98s, HR = 0.5750, NDCG = 0.3234, loss = 0.2989, validated in 4.90s
Iteration 4: 53.73s, HR = 0.6051, NDCG = 0.3381, loss = 0.2930, validated in 5.02s
Iteration 5: 56.02s, HR = 0.6147, NDCG = 0.3495, loss = 0.2856, validated in 5.88s
Iteration 6: 82.54s, HR = 0.6286, NDCG = 0.3569, loss = 0.2812, validated in 8.41s
Iteration 7: 71.14s, HR = 0.6321, NDCG = 0.3582, loss = 0.2803, validated in 11.14s
Iteration 8: 70.75s, HR = 0.6174, NDCG = 0.3510, loss = 0.2801, validated in 10.13s
Iteration 9: 77.31s, HR = 0.6371, NDCG = 0.3621, loss = 0.2802, validated in 12.30s
Iteration 10: 79.50s, HR = 0.6318, NDCG = 0.3586, loss = 0.2807, validated in 11.20s
Iteration 11: 85.42s, HR = 0.6315, NDCG = 0.3579, loss = 